# R-Python-interface-rpy2

#### *Qianjiang Hu*   
###### 2021-02-20

Study notes of `rpy2`  tutorial

In [1]:
import rpy2.situation
for row in rpy2.situation.iter_info():
    print(row)

rpy2 version:
3.3.6
Python version:
3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 23:10:56) 
[GCC 7.3.0]
Looking for R's HOME:
    Environment variable R_HOME: None
    Calling `R RHOME`: /usr/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server
R version:
    In the PATH: R version 4.0.4 (2021-02-15) -- "Lost Library Book"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/usr/share/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'rt', 'dl', 'm', 'icuuc', 'icui18n']
  library_dirs:
  ['/usr/lib/R/lib']
  extra_compile_args:
  []
  extra_link_args:
  ['-Wl,--export-dynamic', '-fopenmp', '-Wl,-Bsymbolic-functions', '-Wl,-z,relro']


In [2]:
## Ignore InputWarning
import warnings
warnings.filterwarnings("ignore")

In [3]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

## Import R Packages

Importing R packages is often the first step when running R code, and `rpy2` is providing a function `rpy2.robjects.packages.importr()` that makes that step very similar to importing Python packages.

In [4]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

# import `ggplot2` package
ggplot2 = importr('ggplot2')

In [5]:
package_names = ('afex', 'emmeans')
type(StrVector(package_names))

rpy2.robjects.vectors.StrVector

In [6]:
### install and import r packages into python

package_names = ('afex', 'emmeans') # packages will be used
base = rpackages.importr('base') # import R's "base" package

if all(rpackages.isinstalled(x) for x in package_names):  # The all() function returns True if all items in an iterable are true, otherwise it returns False. If the iterable object is empty, the all() function also returns True.
    have_package = True
    
else:
    have_package = False
    
if not have_package:
    # import R's "utils" package
    utils = rpackages.importr('utils')
    # R.utils: Various Basic Programming Utilities, including 'install','view' e.g. functions
    utils.chooseCRANmirror(ind = 1)
    
    packnames_to_install = [x for x in package_names if not rpackages.isinstalled(x)]
    
    if len(packnames_to_install) > 0:
        utils.install_packages(StrVector(packnames_to_install))


## import embedded r object

The `__getitem__()` method of rpy2.robjects.r, gets the R object associated with a given symbol, just as typing that symbol name in the R console would do it (see the note below for details).

In [7]:
pi = robjects.r['pi']
print('type of "pi": %s' %(type(pi)))
print('pi = %.6f'%(pi[0]))

type of "pi": <class 'rpy2.robjects.vectors.FloatVector'>
pi = 3.141593


In [8]:
# or
pi = robjects.r('pi')
pi[0]

3.141592653589793

In [9]:
type(pi)

rpy2.robjects.vectors.FloatVector

##### Warning

The result is an R vector rather than a float. The Section R vectors below will provide explanation for the following behavior:

`.r_repr()` String representation for an object that can be directly evaluated as R code.

In [10]:
piplus2 = robjects.r('pi') + 2
piplus2.r_repr()

'c(3.14159265358979, 2)'

In [11]:
# contrast above
pi0plus2 = robjects.r('pi')[0] + 2
print(pi0plus2)

5.141592653589793


## import r objects/function from R global environment

`robjects.r()` can be consider as a R Globle Environment which run the original  R codes and present the R objects

In [12]:
pcr = robjects.r('pcr <- c(1,2,3)')
pcr

1.000000,2.000000,3.000000


Global Environment, that is the place one starts at when in the R console. Whenever the R code creates variables, those variables are “located” in that Global Environment by default.



In [13]:
# use Triple quotes here to write multi-rows and also comments.
robjects.r('''
        # create a function `f`
        f <- function(r, verbose=FALSE) {
            if (verbose) {
                cat("I am calling f().\n")
            }
            2 * pi * r
        }
        # call the function `f` with argument value 3
        f(3)
        ''')

18.849556


import pre-defined r function `f`

In [14]:
r_f = robjects.r['f']

In [15]:
res = r_f(3)
res

18.849556


## Import data

In [16]:
data = robjects.r('read.table(file = "http://personality-project.org/r/datasets/R.appendix3.data",header =T)')
print(data.head())

  Observation Subject Valence Recall
1           1     Jim     Neg     32
2           2     Jim     Neu     15
3           3     Jim     Pos     45
4           4  Victor     Neg     30
5           5  Victor     Neu     13
6           6  Victor     Pos     40



## Creating R vectors

In [17]:
## res.r_repr() # show the r object

In [18]:
#
res = robjects.StrVector(['abc', 'def'])  # create string vector
print('1: ', res.r_repr())
#
res = robjects.IntVector([1, 2, 3])   # create integer vector
print('2: ', res.r_repr())
#
res = robjects.FloatVector([1.1, 2.2, 3.3])    # create Float vector
print('3: ', res.r_repr())

1:  c("abc", "def")
2:  1:3
3:  c(1.1, 2.2, 3.3)


In [19]:
### Create matrix
v = robjects.FloatVector([1.1, 2.2, 3.3, 4.4, 5.5, 6.6])
m = robjects.r['matrix'](v, nrow = 2)
print(m)

     [,1] [,2] [,3]
[1,]  1.1  3.3  5.5
[2,]  2.2  4.4  6.6



##  Calling R function

In [20]:
rsum = robjects.r['sum']
rsum(robjects.IntVector([1,2,3]))[0]

6

In [21]:
rsort = robjects.r['sort']
res = rsort(robjects.IntVector([1,2,3]), decreasing=True)
print(res.r_repr())

3:1


## Graphics and plots

In [22]:
r = robjects.r
x = robjects.IntVector(range(10))
y = r.rnorm(10)

In [23]:
print('x = %s' %(x))
print('x.r_repr() = %s' %(x.r_repr()))

x =  [1] 0 1 2 3 4 5 6 7 8 9

x.r_repr() = 0:9


In [24]:
r.X11() # open an interactive graphics device
r.layout(r.matrix(robjects.IntVector([1,2,3,2]), nrow=2, ncol=2)) # setup figure layout
r.plot(r.runif(10), y, xlab="runif", ylab="foo/bar", col="red") # draw the plot. The plot will show in a new window

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f0eec6c1388> [RTYPES.NILSXP]